<a href="https://colab.research.google.com/github/marc1971/CAS_NLP_final_project/blob/main/Doc_to_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# Connecting to Google Drive to get access to data, that is used for training the model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q -U torch==2.1.0 datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 typing-extensions==4.6.1 pyarrow==14.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1

In [4]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [5]:
def vec_db(dict_path, chunk_size, overlap):
    texts = load_text(dict_path, 'all')
    text_splitter = text_splitting(chunk_size, overlap, separator="")
    documents = create_LangChain_doc(texts, chunk_size, overlap, separator="")
    chunked_documents = chunk_docs(documents,chunk_size, overlap, separator="")
    return chunked_documents

#Text loader

def load_text(dict_path, N):

    '''
    Imports needed:
    - os

    Description:
    Loads ths Text files into a list. The function returns a list of N strings.


    Variables:
    dict_path:      str;  The path of the folder the docs are placed
    N:              int OR str; It is a integer OR the string 'all'. If a integer is found,
                    the corrsponding number of documents are loaded. If 'all is found,
                    all documents are loaded.

    Return: list of str
    '''
    import os
    dict_path = dict_path



    entries = os.listdir(dict_path)
    # Filter out entries that are files
    files = [entry for entry in entries if os.path.isfile(os.path.join(dict_path, entry))]




    N_docs_all = len(files)


    if isinstance(N, int):
        N_docs = N
        # Handle the integer case
        # For example, if N_docs represents a number of documents to process
        # process_integer_docs(N_docs)
    elif isinstance(N, str):
        N_docs = N_docs_all



    texts =[]
    for i in range (1,N_docs+1):
      with open(dict_path+f'/Doc{i:02}.txt', 'r', encoding = 'utf-8') as file:
        texts.append(file.read())

    return texts

#TextSplitter

def text_splitting(chunk_size, overlap, separator=""):
    '''
    Imports needed:
    - from langchain.text_splitter import CharacterTextSplitter

    Description:
    Splitts the text in a certain number of chunks with an overlap of "overlap".
    The Separator should be st to separator=""!
    The function returns a text_splitter.

    Variabels:
    chunk_size:     int; Numer of Characters in the Chunk (i.e. 500)

    Return: function
    '''
    from langchain.text_splitter import CharacterTextSplitter
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size,
                                     chunk_overlap=overlap,
                                     separator= separator
                                     )
    return text_splitter

# HelperFunction to extract Titel of Document
def get_text_before_newline(text):
   '''
    Imports needed:
    - re

    Description:
    Helper Function to read the first Line in each Document, which is the title of the document!
    The Regular Expression is to match any character sequence before the first newline character.
    The function returns a srting with the titel of the document.

    Variabels:
    chunk_size:     int; Numer of Characters in the Chunk (i.e. 500)

    Return: str

    '''
   import re

   match = re.search(r'^(.*?)\n', text, re.DOTALL)

   if match:
    return match.group(1)  # Return the matched group, which is the text before the newline
   else:
      return 'No title found!'

# Creating LangChain Document
def create_LangChain_doc(texts, chunk_size, overlap, separator):
   '''
    Imports needed:
    - from text_loader import get_text_before_newline (Helper Function from above)
    - from text_loder import text_splitting (text_splitter from CharacterTextSplitter is used)

    Description:
    A langchaine-Document with the "create_documents"-methode from TextSplitter is created.
    The TextSplitter uses a list of texts (here: 'texts') and a list of metadatas (here: 'metadatas') to create a list of text chunks
    The helperfunction "get_text_before_newline(text)" is used to read out the title of each document.
    The function returns a langchain-object

    Variabels:
    texts:     list of str; The input variable is a list of strings, with the Titel of the Document in the first line sperated by \n

    Return: LangChain-Object
    '''
   from langchain.text_splitter import CharacterTextSplitter
   metadatas = []

   for text in texts:
      metadatas.append({"document": get_text_before_newline(text)})

   documents = text_splitting(chunk_size, overlap, separator).create_documents(texts, metadatas=metadatas)

   return documents

#Chunk the LangChain Object
def chunk_docs(documents,chunk_size, overlap, separator):
    '''
    Imports needed:
    - from text_loder import text_splitting (text_splitter from CharacterTextSplitter sis used)

    Description:
    The LangChain-Object is split up into the nummer of Chunks, that is defined in the text_splitting function and stored in a List of LangChain-Objects.
    The function returns a List of LangChain-Objects.

    Variabels:
    documents:  LangChain-Object; The input variable is a LangChain-Object

    Return:     List of LangChain-Objects
    '''
    from langchain.text_splitter import CharacterTextSplitter
    chunked_documents = text_splitting(chunk_size, overlap, separator).split_documents(documents)
    return chunked_documents

In [6]:
####################################
# Prepare Documents
###################################

#Set Chunk-Size and Overlap
chunk_size = 500
overlap = 0
dict_path = '/content/drive/My Drive/ColabNotebooks/CAS_NLP_final_project/Textdata/'

texts = load_text(dict_path, 'all')
text_splitter = text_splitting(chunk_size, overlap, separator="")
documents = create_LangChain_doc(texts, chunk_size, overlap, separator="")
chunked_documents = chunk_docs(documents,chunk_size, overlap, separator="")

In [7]:
# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='PM-AI/bi-encoder_msmarco_bert-base_german'))

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/729k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
db.save_local('/content/drive/My Drive/ColabNotebooks/CAS_NLP_final_project/streamlit/documents_index.index')